<a href="https://colab.research.google.com/github/dangcaptkd2/ImageCaptioning/blob/son/ImageCaptioning_Flickr_8k_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##SON


In [1]:
!git clone https://github.com/dangcaptkd2/ImageCaptioning

Cloning into 'ImageCaptioning'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 20 (delta 5), reused 14 (delta 2), pack-reused 0
Unpacking objects: 100% (20/20), done.


In [ ]:
from numpy import array
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import string
import os
import zipfile
from PIL import Image
import glob
from pickle import dump, load
import pickle
from time import time
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import LSTM, Embedding, TimeDistributed, Dense, RepeatVector,\
                         Activation, Flatten, Reshape, concatenate, Dropout, BatchNormalization
                    
from keras.optimizers import Adam, RMSprop
from keras.layers.wrappers import Bidirectional
from keras.layers.merge import add
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras import Input, layers
from keras import optimizers
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

from keras.applications.resnet50 import ResNet50

import keras.preprocessing.image

# Load, preprocess dataset


### Dowload, show, clean caption and understand dataset

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"quyennguyenthinh","key":"52c200c7cee98c6a1dd7c6433c61afd8"}'}

In [ ]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d shadabhussain/flickr8k

100% 2.13G/2.13G [00:25<00:00, 37.8MB/s]
100% 2.13G/2.13G [00:25<00:00, 88.4MB/s]


In [ ]:
with zipfile.ZipFile("/content/flickr8k.zip","r") as zip_ref:
    zip_ref.extractall("/content/")

In [ ]:
# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

filename = "/content/flickr_data/Flickr_Data/Flickr_TextData/Flickr8k.token.txt"
# load descriptions
doc = load_doc(filename)
print(doc[:300])

1000268201_693b08cb0e.jpg#0	A child in a pink dress is climbing up a set of stairs in an entry way .
1000268201_693b08cb0e.jpg#1	A girl going into a wooden building .
1000268201_693b08cb0e.jpg#2	A little girl climbing into a wooden playhouse .
1000268201_693b08cb0e.jpg#3	A little girl climbing the s


Load description as a dictionary

In [ ]:
def load_descriptions(doc):
	mapping = dict()
	# process lines
	for line in doc.split('\n'):
		# split line by white space
		tokens = line.split()
		if len(line) < 2:
			continue
		# take the first token as the image id, the rest as the description
		image_id, image_desc = tokens[0], tokens[1:]
		# extract filename from image id
		image_id = image_id.split('.')[0]
		# convert description tokens back to string
		image_desc = ' '.join(image_desc)
		# create the list if needed
		if image_id not in mapping:
			mapping[image_id] = list()
		# store description
		mapping[image_id].append(image_desc)
	return mapping

# parse descriptions
descriptions = load_descriptions(doc)
print('Loaded: %d ' % len(descriptions))

Loaded: 8092 


In [ ]:
descriptions['1000268201_693b08cb0e']

['A child in a pink dress is climbing up a set of stairs in an entry way .',
 'A girl going into a wooden building .',
 'A little girl climbing into a wooden playhouse .',
 'A little girl climbing the stairs to her playhouse .',
 'A little girl in a pink dress going into a wooden cabin .']

Clean caption

In [ ]:
def clean_descriptions(descriptions):
	# prepare translation table for removing punctuation
	table = str.maketrans('', '', string.punctuation)
	for key, desc_list in descriptions.items():
		for i in range(len(desc_list)):
			desc = desc_list[i]
			# tokenize
			desc = desc.split()
			# convert to lower case
			desc = [word.lower() for word in desc]
			# remove punctuation from each token
			desc = [w.translate(table) for w in desc]
			# remove hanging 's' and 'a'
			desc = [word for word in desc if len(word)>1]
			# remove tokens with numbers in them
			desc = [word for word in desc if word.isalpha()]
			# store as string
			desc_list[i] =  ' '.join(desc)

# clean descriptions
clean_descriptions(descriptions)

In [ ]:
# convert the loaded descriptions into a vocabulary of words
def to_vocabulary(descriptions):
	# build a list of all description strings
	all_desc = set()
	for key in descriptions.keys():
		[all_desc.update(d.split()) for d in descriptions[key]]
	return all_desc

# summarize vocabulary
vocabulary = to_vocabulary(descriptions)
print('Original Vocabulary Size: %d' % len(vocabulary))

Original Vocabulary Size: 8763


In [ ]:
# save descriptions to file, one per line
def save_descriptions(descriptions, filename):
	lines = list()
	for key, desc_list in descriptions.items():
		for desc in desc_list:
			lines.append(key + ' ' + desc)
	data = '\n'.join(lines)
	file = open(filename, 'w')
	file.write(data)
	file.close()

save_descriptions(descriptions, 'descriptions.txt')

In [ ]:
# load a pre-defined list of photo identifiers
def load_set(filename):
	doc = load_doc(filename)
	dataset = list()
	# process line by line
	for line in doc.split('\n'):
		# skip empty lines
		if len(line) < 1:
			continue
		# get the image identifier
		identifier = line.split('.')[0]
		dataset.append(identifier)
	return set(dataset)

# load training dataset (6K)
filename = '/content/flickr_data/Flickr_Data/Flickr_TextData/Flickr_8k.trainImages.txt'
train = load_set(filename)
print('Dataset train: %d' % len(train))

filename = '/content/flickr_data/Flickr_Data/Flickr_TextData/Flickr_8k.testImages.txt'
test = load_set(filename)
print('Dataset test: %d' % len(test))

Dataset train: 6000
Dataset test: 1000


In [ ]:
# Below path contains all the images
images = '/content/flickr_data/Flickr_Data/Images/'
# Create a list of all image names in the directory
img = glob.glob(images + '*.jpg')

In [ ]:
# Below file conatains the names of images to be used in train data
train_images_file = '/content/flickr_data/Flickr_Data/Flickr_TextData/Flickr_8k.trainImages.txt'
test_images_file = '/content/flickr_data/Flickr_Data/Flickr_TextData/Flickr_8k.testImages.txt'
# Read the train image names in a set
train_images = set(open(train_images_file, 'r').read().strip().split('\n'))
test_images = set(open(test_images_file, 'r').read().strip().split('\n'))

# Create a list of all the training images with their full path names
train_img = []
test_img= []

for i in img: # img is list of full path names of all images
    if i[len(images):] in train_images: # Check if the image belongs to training set
        train_img.append(i) # Add it to the list of train images

for i in img: # img is list of full path names of all images
    if i[len(images):] in test_images: # Check if the image belongs to training set
        test_img.append(i) # Add it to the list of train images

In [ ]:
len(train_img)

6000

In [ ]:
train_img[:5]

['/content/flickr_data/Flickr_Data/Images/2663248626_f000f2661d.jpg',
 '/content/flickr_data/Flickr_Data/Images/2088460083_42ee8a595a.jpg',
 '/content/flickr_data/Flickr_Data/Images/2473293833_78820d2eaa.jpg',
 '/content/flickr_data/Flickr_Data/Images/3550459890_161f436c8d.jpg',
 '/content/flickr_data/Flickr_Data/Images/2619454551_c4bb726a85.jpg']

In [ ]:
# Below file conatains the names of images to be used in test data
test_images_file = '/content/flickr_data/Flickr_Data/Flickr_TextData/Flickr_8k.testImages.txt'
# Read the validation image names in a set# Read the test image names in a set
test_images = set(open(test_images_file, 'r').read().strip().split('\n'))

# Create a list of all the test images with their full path names
test_img = []

for i in img: # img is list of full path names of all images
    if i[len(images):] in test_images: # Check if the image belongs to test set
        test_img.append(i) # Add it to the list of test images

In [ ]:
len(test_img)

1000

In [ ]:
# load clean descriptions into memory
def load_clean_descriptions(filename, dataset):
	# load document
	doc = load_doc(filename)
	descriptions = dict()
	for line in doc.split('\n'):
		# split line by white space
		tokens = line.split()
		# split id from description
		image_id, image_desc = tokens[0], tokens[1:]
		# skip images not in the set
		if image_id in dataset:
			# create list
			if image_id not in descriptions:
				descriptions[image_id] = list()
			# wrap description in tokens
			desc = 'startseq ' + ' '.join(image_desc) + ' endseq'
			# store
			descriptions[image_id].append(desc)
	return descriptions

# descriptions
train_descriptions = load_clean_descriptions('descriptions.txt', train)
print('Descriptions: train=%d' % len(train_descriptions))

Descriptions: train=6000


In [ ]:
test_descriptions = load_clean_descriptions('descriptions.txt', test)
print('Descriptions: test=%d' % len(test_descriptions))

Descriptions: test=1000


# Data preprocessing - Images

In [ ]:
def preprocess(image_path):
    # Convert all the images to size 299x299 as expected by the inception v3 model
    img = keras.preprocessing.image.load_img(image_path, target_size=(299, 299))
    # Convert PIL image to numpy array of 3-dimensions
    x = keras.preprocessing.image.img_to_array(img)
    # Add one more dimension
    x = np.expand_dims(x, axis=0)
    # preprocess the images using preprocess_input() from inception module
    x = preprocess_input(x)
    return x

In [ ]:
# Load the inception v3 model
model = InceptionV3(weights='imagenet')

96116736/96112376 [==============================] - 1s 0us/step


In [ ]:
# Create a new model, by removing the last layer (output layer) from the inception v3
model_new = Model(model.input, model.layers[-2].output)

In [ ]:
# Function to encode a given image into a vector of size (2048, )
def encode(image_path):
    image = preprocess(image_path) # preprocess the image
    fea_vec = model_new.predict(image) # Get the encoding vector for the image
    fea_vec = np.reshape(fea_vec, fea_vec.shape[1]) # reshape from (1, 2048) to (2048, )
    return fea_vec

In [ ]:
# Call the funtion to encode all the train images
# This will take a while on CPU - Execute this only once
"""start = time()
encoding_train = {}
for img in train_img:
    encoding_train[img[len(images):]] = encode(img)
print("Time taken in seconds =", time()-start)"""

'start = time()\nencoding_train = {}\nfor img in train_img:\n    encoding_train[img[len(images):]] = encode(img)\nprint("Time taken in seconds =", time()-start)'

In [ ]:
# Save the bottleneck train features to disk
#with open("/content/drive/My Drive/Image captioning/encoded_train_images_3.pkl", "wb") as encoded_pickle:
 #   pickle.dump(encoding_train, encoded_pickle)

In [ ]:
# Call the funtion to encode all the test images - Execute this only once
"""start = time()
encoding_test = {}
for img in test_img:
    encoding_test[img[len(images):]] = encode(img)
print("Time taken in seconds =", time()-start)"""

'start = time()\nencoding_test = {}\nfor img in test_img:\n    encoding_test[img[len(images):]] = encode(img)\nprint("Time taken in seconds =", time()-start)'

In [ ]:
train_features = load(open("/content/drive/My Drive/Image captioning/encoded_train_images_3.pkl", "rb"))
print('Photos: train=%d' % len(train_features))

Photos: train=6000


In [ ]:
test_features = load(open("/content/drive/MyDrive/Image captioning/encoded_test_images_3.pkl", "rb"))
print('Photos: test=%d' % len(test_features))

Photos: test=1000


# Data preprocessing - caption

In [ ]:
# Create a list of all the training captions
all_train_captions = []
for key, val in train_descriptions.items():
    for cap in val:
        all_train_captions.append(cap)
len(all_train_captions)

30000

In [ ]:
#Create a list of all the testing captions
all_test_captions = []
for key, val in test_descriptions.items():
    for cap in val:
        all_test_captions.append(cap)
len(all_test_captions)

5000

In [ ]:
# Consider only words which occur at least 10 times in the corpus
word_count_threshold = 10
word_counts = {}
nsents = 0
for sent in all_train_captions:
    nsents += 1
    for w in sent.split(' '):
        word_counts[w] = word_counts.get(w, 0) + 1

vocab = [w for w in word_counts if word_counts[w] >= word_count_threshold]
print('preprocessed words %d -> %d' % (len(word_counts), len(vocab)))

preprocessed words 7578 -> 1651


In [ ]:
ixtoword = {}
wordtoix = {}

ix = 1
for w in vocab:
    wordtoix[w] = ix
    ixtoword[ix] = w
    ix += 1

In [ ]:
vocab_size = len(ixtoword) + 1 # one for appended 0's
vocab_size

1652

In [ ]:
# Consider only words which occur at least 10 times in the corpus
word_count_threshold = 10
word_counts = {}
nsents = 0
for sent in all_test_captions:
    nsents += 1
    for w in sent.split(' '):
        word_counts[w] = word_counts.get(w, 0) + 1

vocab_test = [w for w in word_counts if word_counts[w] >= word_count_threshold]
print('preprocessed words TEST %d -> %d' % (len(word_counts), len(vocab_test)))

preprocessed words TEST 3178 -> 526


In [ ]:
ixtoword_test = {}
wordtoix_test = {}

ix = 1
for w in vocab_test:
    wordtoix_test[w] = ix
    ixtoword_test[ix] = w
    ix += 1

In [ ]:
vocab_size_test = len(ixtoword_test) + 1 # one for appended 0's
vocab_size_test

527

In [ ]:
# convert a dictionary of clean descriptions to a list of descriptions
def to_lines(descriptions):
	all_desc = list()
	for key in descriptions.keys():
		[all_desc.append(d) for d in descriptions[key]]
	return all_desc

# calculate the length of the description with the most words
def max_length(descriptions):
	lines = to_lines(descriptions)
	return max(len(d.split()) for d in lines)

# determine the maximum sequence length
max_length = max_length(train_descriptions)
print('Description Length: %d' % max_length)

Description Length: 34


In [ ]:
train_descriptions['1000268201_693b08cb0e']

['startseq child in pink dress is climbing up set of stairs in an entry way endseq',
 'startseq girl going into wooden building endseq',
 'startseq little girl climbing into wooden playhouse endseq',
 'startseq little girl climbing the stairs to her playhouse endseq',
 'startseq little girl in pink dress going into wooden cabin endseq']

In [ ]:
# data generator, intended to be used in a call to model.fit_generator()
def data_generator(descriptions, photos, wordtoix, max_length, num_photos_per_batch):
    X1, X2, y = list(), list(), list()
    n=0
    # loop for ever over images
    while 1:
        for key, desc_list in descriptions.items():
            n+=1
            # retrieve the photo feature
            photo = photos[key+'.jpg']
            for desc in desc_list:
                # encode the sequence
                seq = [wordtoix[word] for word in desc.split(' ') if word in wordtoix]
                # split one sequence into multiple X, y pairs
                for i in range(1, len(seq)):
                    # split into input and output pair
                    in_seq, out_seq = seq[:i], seq[i]
                    # pad input sequence
                    in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
                    # encode output sequence
                    out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
                    # store
                    X1.append(photo)
                    X2.append(in_seq)
                    y.append(out_seq)
            # yield the batch data
            if n==num_photos_per_batch:
                yield ([array(X1), array(X2)], array(y))
                X1, X2, y = list(), list(), list()
                n=0

In [ ]:
with zipfile.ZipFile("/content/drive/My Drive/Image captioning/glove.6B.zip","r") as zip_ref:
    zip_ref.extractall("/content/")

In [ ]:
embeddings_index = {} # empty dictionary
f = open('/content/glove.6B.200d.txt', encoding="utf-8")

for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [ ]:
embeddings_index['the']

array([-7.1549e-02,  9.3459e-02,  2.3738e-02, -9.0339e-02,  5.6123e-02,
        3.2547e-01, -3.9796e-01, -9.2139e-02,  6.1181e-02, -1.8950e-01,
        1.3061e-01,  1.4349e-01,  1.1479e-02,  3.8158e-01,  5.4030e-01,
       -1.4088e-01,  2.4315e-01,  2.3036e-01, -5.5339e-01,  4.8154e-02,
        4.5662e-01,  3.2338e+00,  2.0199e-02,  4.9019e-02, -1.4132e-02,
        7.6017e-02, -1.1527e-01,  2.0060e-01, -7.7657e-02,  2.4328e-01,
        1.6368e-01, -3.4118e-01, -6.6070e-02,  1.0152e-01,  3.8232e-02,
       -1.7668e-01, -8.8153e-01, -3.3895e-01, -3.5481e-02, -5.5095e-01,
       -1.6899e-02, -4.3982e-01,  3.9004e-02,  4.0447e-01, -2.5880e-01,
        6.4594e-01,  2.6641e-01,  2.8009e-01, -2.4625e-02,  6.3302e-01,
       -3.1700e-01,  1.0271e-01,  3.0886e-01,  9.7792e-02, -3.8227e-01,
        8.6552e-02,  4.7075e-02,  2.3511e-01, -3.2127e-01, -2.8538e-01,
        1.6670e-01, -4.9707e-03, -6.2714e-01, -2.4904e-01,  2.9713e-01,
        1.4379e-01, -1.2325e-01, -5.8178e-02, -1.0290e-03, -8.21

In [ ]:
embedding_dim = 200

# Get 200-dim dense vector for each of the 10000 words in out vocabulary
embedding_matrix = np.zeros((vocab_size, embedding_dim))

for word, i in wordtoix.items():
    #if i < max_words:
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in the embedding index will be all zeros
        embedding_matrix[i] = embedding_vector

In [ ]:
embedding_matrix.shape

(1652, 200)

# Model architecture

In [ ]:
#max_length = 34
#vocab_size = 7579

In [ ]:
inputs1 = Input(shape=(2048,))
fe1 = Dropout(0.5)(inputs1)
fe2 = Dense(256, activation='relu')(fe1)

inputs2 = Input(shape=(max_length,))
se1 = Embedding(vocab_size, embedding_dim, mask_zero=True)(inputs2)
se2 = Dropout(0.5)(se1)
se3 = LSTM(256)(se2)

decoder1 = add([fe2, se3])
decoder2 = Dense(256, activation='relu')(decoder1)
outputs = Dense(vocab_size, activation='softmax')(decoder2)
model = Model(inputs=[inputs1, inputs2], outputs=outputs)

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 34)]         0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            [(None, 2048)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 34, 200)      330400      input_2[0][0]                    
__________________________________________________________________________________________________
dropout (Dropout)               (None, 2048)         0           input_1[0][0]                    
______________________________________________________________________________________________

In [ ]:
model.layers[2]

In [ ]:
model.layers[2].set_weights([embedding_matrix])
model.layers[2].trainable = False

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
epochs = 200
number_pics_per_bath = 6
steps = len(train_descriptions)//number_pics_per_bath

In [ ]:
# for i in range(epochs):
#     generator = data_generator(train_descriptions, train_features, wordtoix, max_length, number_pics_per_bath)
#     generator_valid = data_generator(test_descriptions, test_features, wordtoix_test, max_length, number_pics_per_bath)
#     model.fit(generator, epochs=1, steps_per_epoch=steps, verbose=1, 
#                         validation_data=generator_valid, validation_steps =len(test_descriptions)//number_pics_per_bath)
#     #model.save('/content/drive/My Drive/Image captioning/input_output/model_' + str(i) + '.h5')

In [ ]:
#generator = data_generator(train_descriptions, train_features, wordtoix, max_length, number_pics_per_bath)
#generator_valid = data_generator(test_descriptions, test_features, wordtoix_test, max_length, number_pics_per_bath)
history = model.fit(data_generator(train_descriptions, train_features, wordtoix, max_length, number_pics_per_bath), 
                    epochs=epochs, steps_per_epoch=steps, verbose=1, 
                    validation_data=data_generator(test_descriptions, test_features, wordtoix_test, max_length, number_pics_per_bath), 
                    validation_steps =len(test_descriptions)//number_pics_per_bath)
#model.save('/content/drive/My Drive/Image captioning/input_output/model_' + str(i) + '.h5')

Epoch 1/200
1000/1000 [==============================] - 101s 98ms/step - loss: 4.7250 - accuracy: 0.1926 - val_loss: 8.0321 - val_accuracy: 0.0080
Epoch 2/200
1000/1000 [==============================] - 96s 96ms/step - loss: 3.5662 - accuracy: 0.2796 - val_loss: 8.7353 - val_accuracy: 0.0074
Epoch 3/200
1000/1000 [==============================] - 99s 99ms/step - loss: 3.2899 - accuracy: 0.3052 - val_loss: 9.1658 - val_accuracy: 0.0079
Epoch 4/200
1000/1000 [==============================] - 99s 99ms/step - loss: 3.1296 - accuracy: 0.3199 - val_loss: 9.4273 - val_accuracy: 0.0080
Epoch 5/200
1000/1000 [==============================] - 98s 98ms/step - loss: 3.0159 - accuracy: 0.3303 - val_loss: 9.6874 - val_accuracy: 0.0078
Epoch 6/200
1000/1000 [==============================] - 98s 98ms/step - loss: 2.9314 - accuracy: 0.3397 - val_loss: 10.0162 - val_accuracy: 0.0078
Epoch 7/200
1000/1000 [==============================] - 98s 98ms/step - loss: 2.8603 - accuracy: 0.3454 - val_loss:

In [ ]:
val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Best epoch: 1


In [ ]:
# model.optimizer.lr = 0.0001
# epochs = 3
# number_pics_per_bath = 6
# steps = len(train_descriptions)//number_pics_per_bath

In [ ]:
# hist =model.fit_generator(data_generator(train_descriptions, train_features, wordtoix, max_length, number_pics_per_bath), 
#                           epochs=epochs, steps_per_epoch=steps, verbose=1,)
#     #model.save('./model_weights/model_' + str(i) + '.h5')

In [ ]:
def plot_model_history(model_history, acc='accuracy', val_acc='val_accuracy'):
    fig, axs = plt.subplots(1,2,figsize=(15,5))
    axs[0].plot(range(1,len(model_history.history[acc])+1),model_history.history[acc])
    axs[0].plot(range(1,len(model_history.history[val_acc])+1),model_history.history[val_acc])
    axs[0].set_title('Model Accuracy')
    axs[0].set_ylabel('Accuracy')
    axs[0].set_xlabel('Epoch')
    axs[0].set_xticks(np.arange(1,len(model_history.history[acc])+1),len(model_history.history[acc])/10)
    axs[0].legend(['train', 'val'], loc='best')
    axs[1].plot(range(1,len(model_history.history['loss'])+1),model_history.history['loss'])
    axs[1].plot(range(1,len(model_history.history['val_loss'])+1),model_history.history['val_loss'])
    axs[1].set_title('Model Loss')
    axs[1].set_ylabel('Loss')
    axs[1].set_xlabel('Epoch')
    axs[1].set_xticks(np.arange(1,len(model_history.history['loss'])+1),len(model_history.history['loss'])/10)
    axs[1].legend(['train', 'val'], loc='best')
    plt.show()
plot_model_history(hist)


In [ ]:
model.save_weights('/content/drive/My Drive/Image captioning/input_output/model_31.h5')

In [ ]:
model.load_weights('/content/drive/My Drive/Image captioning/input_output/model_30.h5')

In [ ]:
with open("/content/drive/My Drive/Image captioning/encoded_test_images_3.pkl", "rb") as encoded_pickle:
    encoding_test = load(encoded_pickle)

In [ ]:
def greedySearch(photo):
    in_text = 'startseq'
    for i in range(max_length):
        sequence = [wordtoix[w] for w in in_text.split() if w in wordtoix]
        sequence = pad_sequences([sequence], maxlen=max_length)
        yhat = model.predict([photo,sequence], verbose=0)
        #print(yhat.shape)
        yhat = np.argmax(yhat)
        word = ixtoword[yhat]
        in_text += ' ' + word
        if word == 'endseq':
            break
    final = in_text.split()
    final = final[1:-1]
    final = ' '.join(final)
    return final

In [ ]:
images = '/content/flickr_data/Flickr_Data/Images/'

In [ ]:
import random
for i in range(10):
  z=random.randint(1, len(encoding_test.keys()))
  pic = list(encoding_test.keys())[z]
  image = encoding_test[pic].reshape((1,2048))
  x=plt.imread(images+pic)
  plt.imshow(x)
  plt.show()
  print("Greedy:",greedySearch(image))

# Encode new image and generator caption

In [ ]:
img_path = '/content/quyendeptrai.jpg'
feature_vector = encode(img_path)
feature_vector.shape

In [ ]:
new_fea_vec = feature_vector.reshape((1,2048))

In [ ]:
x=plt.imread(img_path)
plt.imshow(x)
plt.show()
print("Greedy:",greedySearch(new_fea_vec))

In [ ]:
def fast_predict(img_path):
  feature_vector = encode(img_path)
  new_fea_vec = feature_vector.reshape((1,2048))
  x=plt.imread(img_path)
  plt.imshow(x)
  plt.show()
  print("Description:",greedySearch(new_fea_vec))

In [ ]:
fast_predict('/content/quyendeptrai2.jpg')

In [ ]:
fast_predict('/content/fourdogs.jpg')

In [ ]:
fast_predict('/content/bigfish.jpg')

In [ ]:
from nltk.translate.bleu_score import sentence_bleu
reference = [['this', 'is', 'a', 'test', 'hello'], ['this', 'is' 'test']]
candidate = ['this', 'is', 'a', 'test']
score = sentence_bleu(reference, candidate)
print(score)

0.7788007830714049


# resnet

In [ ]:
model_2=ResNet50(include_top=False, weights='imagenet',pooling='avg',input_shape=(224,224,3))
model_2.summary()

In [ ]:
import numpy as np
dick = {}
dick[1] = [np.array([1,2,3,4]),np.array([1,2,3,4]),np.array([1,2,3,4]),np.array([1,2,3,4])]
dick[2] = [np.array([1,2,3,4]),np.array([1,2,3,4]),np.array([1,2,3,4]),np.array([1,2,3,4])]
dick[3] = [np.array([1,2,3,4]),np.array([1,2,3,4]),np.array([1,2,3,4]),np.array([1,2,3,4])]
dick

In [ ]:
for key, val in dick.items():
  total = []
  for cc in val:
    t = []
    t.extend(cc)
    total.append(t)
  dick[key] = total
dick

In [ ]:
s  = input()
arr = []
while(1):
  n=input()
  if n == '.' :
    break
  else : arr.append(n) 
